# eda-us-accidents-2016-to-2022

Use the "Run" button to execute the code.

In [2]:
!pip install jovian --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
import jovian

In [4]:
# Execute this to save new versions of the notebook
jovian.commit(project="eda-us-accidents-2016-to-2022")

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.


# US Accidents Exploratory Analysis


1.   ToDo - Talk about EDA
2.   ToDo - Talk about dataset (source, what it contents, how it will be useful)
  - Kaggle
  - Information about accidents
  - Can be useful to prevent future accidents?
  - Mention that this dataset does not contain information about New York.
    



## Download dataset

In [ ]:
pip install opendatasets --upgrade --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import opendatasets as od
# dataset_url = 'https://www.kaggle.com/datasets/sobhanmoosavi/us-accidents'
# od.download(dataset_url)

In [ ]:
import zipfile
from google.colab import drive

drive.mount('/content/drive/')

zip_ref = zipfile.ZipFile("/content/drive/MyDrive/Jovian/prachisarode95 eda-us-accidents-2016-to-2022/archive.zip", 'r')
zip_ref.extractall("/us_acc")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!unzip -q "/content/drive/MyDrive/Jovian/prachisarode95 eda-us-accidents-2016-to-2022/archive.zip"

In [5]:
data_filename = '/content/US_Accidents_March23.csv'

## Data Preparation and Cleaning
- Load the file using Pandas
- Look at some information about the data & the columns
- Fix missing or incorrect values

In [6]:
import pandas as pd

In [8]:
# df = pd.read_csv(data_filename)

In [ ]:
df.columns

In [ ]:
len(df.columns)

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# numeric_df = df.select_dtypes(include=numerics)
# len(numeric_df.columns)

Percentage of missing values per column

In [ ]:
missing_percentages = df.isna().sum().sort_values(ascending=False) / len(df)
missing_percentages

In [ ]:
missing_percentages[missing_percentages != 0 ]

Remove columns that you don't want to use

In [ ]:
missing_percentages[missing_percentages != 0 ].plot(kind='barh')

# Exploratory Analysis and Visualization
Columns we'll analyse:
  1. City
  2. Start Time
  3. Start Lat & Start Lng
  4. Temperature
  5. Weather Condition

In [ ]:
df.columns

In [ ]:
df.City

In [ ]:
cities = df.City.unique()
len(cities)

In [ ]:
cities_by_accidents =  df.City.value_counts()
cities_by_accidents

In [ ]:
cities_by_accidents[:10]

In [ ]:
'NY' in df.State

In [ ]:
cities_by_accidents[:20].plot(kind='barh')

Visualization showing distribution of the higher number of accidents 

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")

In [ ]:
sns.distplot(cities_by_accidents, log_scale=True )

In [ ]:
high_accident_cities = cities_by_accidents[cities_by_accidents >= 1000]
low_accident_cities = cities_by_accidents[cities_by_accidents < 1000]

In [ ]:
len(high_accident_cities) / len(cities)
len(low_accident_cities) / len(cities)

In [ ]:
sns.distplot(high_accident_cities)

In [ ]:
sns.distplot(low_accident_cities)

In [ ]:
cities_by_accidents[cities_by_accidents == 1]

Analyse the time accidents happened

In [ ]:
df.Start_Time = pd.to_datetime(df.Start_Time)

In [ ]:
df.Start_Time[0]

Figure out how to show percentages

In [ ]:
sns.histplot(df.Start_Time.dt.hour / len(df), bins=24, kde=False)

- A high percentage of accidents occur between 6 am to 10 am (probably people tend to be in hurry)
- Next highest percentage is 3 pm to 6 pm 

In [ ]:
sns.histplot(df.Start_Time.dt.dayofweek, bins=7, kde=False)

Is the distribution of accidents by hour the same on weekends as on weekdays?

In [ ]:
df.Start_Time[df.Start_Time.dt.dayofweek == 6] or df.Start_Time[df.Start_Time.dt.dayofweek == 5]

In [ ]:
sunday_start_time = df.Start_Time[df.Start_Time.dt.dayofweek == 6]
sns.distplot(sunday_start_time.dt.hour, bins=24, kde=False, norm_hist=True)

In [ ]:
monday_start_time = df.Start_Time[df.Start_Time.dt.dayofweek == 0]
sns.distplot(monday_start_time.dt.hour, bins=24, kde=False, norm_hist=True)

On sundays, the peak occurs between 10 am and 3 pm, unlike weekdays

In [ ]:
df.Start_Time.dt.year

In [ ]:
df_2019 = df[df.Start_Time.dt.year == 2019]
df_2019_Bing = df_2019[df_2019.Source == 'Bing']
# df_2019_Mapquest = df_2019[df_2019.Source == 'Mapquest']
sns.distplot(df_2019.Start_Time.dt.month, bins=12, kde=False, norm_hist=True)

Can you explain the month-wise trend of accidents?
- Much data is missing for 2016 or maybe even for 2017. 
- There seems to be some issue with the Bing data

In [ ]:
df.Source.value_counts().plot(kind='pie')

Consider excluding bing data, seems to have issues with it.

## Start Latitude and Longitude

In [ ]:
df.Start_Lat

In [ ]:
df.Start_Lng

In [ ]:
# taking a small sample out of the entire dataset
sample_df = df.sample(int(0.1 * len(df)))
sns.scatterplot(x=sample_df.Start_Lng, y=sample_df.Start_Lat, size=0.001)

Visualize data on the map using folium library

In [ ]:
import folium
lat, lng = df.Start_Lat[0], df.Start_Lng[0]
lat, lng

In [ ]:
map = folium.Map(location=[38.9, -77.05], zoom_start=12)
marker = folium.Marker((lat, lng))
marker.add_to(map)
map

In [ ]:
# for x in df[['Start_Lat', 'Start_Lng']].sample(100).iteritems():
#   print(x)

In [ ]:
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
# create a heat map
sample_df = df.sample(int(0.001 * len(df)))
paired_lat_lngs = list(zip(list(sample_df.Start_Lat), list(sample_df.Start_Lng))) #will give you a list of pairs of lat lng values
 
HeatMap(paired_lat_lngs).add_to(map)
map

Perform further analysis according to the remaining questions..

## Ask & Answer Questions
 1. Are there more accidents in warmer or colder areas?
 2. Which 5 states have the highest number of accidents? How about per capita?
 3. Does New York show up in the data? If yes, why is the count lower if it's the most populated city?
 4. Among the top 100 cities in the number of accidents, which states do they belong to most frequently?
 5. What time of the day are accidents most frequent in?
 6. Which days of the week have the most accidents?
 7. Which months have the most accidents?
 8. What is the trend of accidents year over year(decreasing/increasing)?
 9. When is the accidents per unit of traffic the highest?

## Summary & Conclusion

Insights:
  - No data from the New York city
  - The number of accidents per city follows an exponential decrease.
  - Less that 5% of cities have more than 1000 yearly accidents
  - Over 1200 cities have reported just one accident

In [9]:
jovian.commit()

[jovian] Detected Colab notebook...
[jovian] jovian.commit() is no longer required on Google Colab. If you ran this notebook from Jovian, 
then just save this file in Colab using Ctrl+S/Cmd+S and it will be updated on Jovian. 
Also, you can also delete this cell, it's no longer necessary.
